<a href="https://colab.research.google.com/github/vignesh-pala/NLP/blob/master/NLP_Ch14_TextGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Challenge 14 (NLP with Keras-TF2)**

 

Learn how to generate text using LSTM and use it with your favorite ebook to generate some meaningful sentences.

https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

 

You can use Google Colab for the actual coding and submit the link in this yammer thread.....

-----------------End------------

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

# Loss function
from tensorflow.keras.losses import sparse_categorical_crossentropy

from tensorflow.keras.models import load_model

In [2]:
path_to_file = '/content/drive/My Drive/Colab Notebooks/TextFiles/Jungle_Book.txt'

In [3]:
text = open(path_to_file,'r').read()

In [4]:
print(text[:1000])

﻿It was seven o’clock of a very warm evening in the Seeonee hills when
Father Wolf woke up from his day’s rest, scratched himself, yawned, and
spread out his paws one after the other to get rid of the sleepy feeling
in their tips. Mother Wolf lay with her big gray nose dropped across her
four tumbling, squealing cubs, and the moon shone into the mouth of the
cave where they all lived. “Augrh!” said Father Wolf. “It is time to
hunt again.” He was going to spring down hill when a little shadow with
a bushy tail crossed the threshold and whined: “Good luck go with you, O
Chief of the Wolves. And good luck and strong white teeth go with noble
children that they may never forget the hungry in this world.”

It was the jackal--Tabaqui, the Dish-licker--and the wolves of India
despise Tabaqui because he runs about making mischief, and telling
tales, and eating rags and pieces of leather from the village
rubbish-heaps. But they are afraid of him too, because Tabaqui, more
than anyone else in th

Get Unique chars

In [5]:
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']


86

Preprocessing

In [6]:
char_to_ind = {u:i for i,u in  enumerate(vocab)}

In [7]:
print(char_to_ind)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '%': 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, '@': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, '[': 52, ']': 53, '`': 54, 'a': 55, 'b': 56, 'c': 57, 'd': 58, 'e': 59, 'f': 60, 'g': 61, 'h': 62, 'i': 63, 'j': 64, 'k': 65, 'l': 66, 'm': 67, 'n': 68, 'o': 69, 'p': 70, 'q': 71, 'r': 72, 's': 73, 't': 74, 'u': 75, 'v': 76, 'w': 77, 'x': 78, 'y': 79, 'z': 80, '‘': 81, '’': 82, '“': 83, '”': 84, '\ufeff': 85}


In [8]:
ind_to_char = np.array(vocab)

In [9]:
ind_to_char

array(['\n', ' ', '!', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0',
       '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
       'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w',
       'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff'], dtype='<U1')

In [10]:
ind_to_char[30]

'E'

In [11]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [12]:
print(encoded_text)

[85 34 74 ... 73 10  0]


In [13]:
sample = text[:20]
sample

'\ufeffIt was seven o’cloc'

In [14]:
encoded_text[:20]

array([85, 34, 74,  1, 77, 55, 73,  1, 73, 59, 76, 59, 68,  1, 69, 82, 57,
       66, 69, 57])

In [15]:
encoded_text.shape

(291664,)

In [16]:
print(text[:1000])

﻿It was seven o’clock of a very warm evening in the Seeonee hills when
Father Wolf woke up from his day’s rest, scratched himself, yawned, and
spread out his paws one after the other to get rid of the sleepy feeling
in their tips. Mother Wolf lay with her big gray nose dropped across her
four tumbling, squealing cubs, and the moon shone into the mouth of the
cave where they all lived. “Augrh!” said Father Wolf. “It is time to
hunt again.” He was going to spring down hill when a little shadow with
a bushy tail crossed the threshold and whined: “Good luck go with you, O
Chief of the Wolves. And good luck and strong white teeth go with noble
children that they may never forget the hungry in this world.”

It was the jackal--Tabaqui, the Dish-licker--and the wolves of India
despise Tabaqui because he runs about making mischief, and telling
tales, and eating rags and pieces of leather from the village
rubbish-heaps. But they are afraid of him too, because Tabaqui, more
than anyone else in th

In [17]:
len('﻿It was seven o’clock of a very warm evening in the Seeonee hills when')

70

In [18]:
len(''' 
It was seven o’clock of a very warm evening in the Seeonee hills when
Father Wolf woke up from his day’s rest, scratched himself, yawned, and
spread out his paws one after the other to get rid of the sleepy feeling
''')

217

In [19]:
seq_len = 100
total_num_seq =  len(text)//(seq_len + 1)
total_num_seq

2887

In [20]:
  # Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])

﻿
I
t
 
w
a
s
 
s
e
v
e
n
 
o
’
c
l
o
c
k
 
o
f
 
a
 
v
e
r
y
 
w
a
r
m
 
e
v
e
n
i
n
g
 
i
n
 
t
h
e
 
S
e
e
o
n
e
e
 
h
i
l
l
s
 
w
h
e
n


F
a
t
h
e
r
 
W
o
l
f
 
w
o
k
e
 
u
p
 
f
r
o
m
 
h
i
s
 
d
a
y
’
s
 
r
e
s
t
,
 
s
c
r
a
t
c
h
e
d
 
h
i
m
s
e
l
f
,
 
y
a
w
n
e
d
,
 
a
n
d


s
p
r
e
a
d
 
o
u
t
 
h
i
s
 
p
a
w
s
 
o
n
e
 
a
f
t
e
r
 
t
h
e
 
o
t
h
e
r
 
t
o
 
g
e
t
 
r
i
d
 
o
f
 
t
h
e
 
s
l
e
e
p
y
 
f
e
e
l
i
n
g


i
n
 
t
h
e
i
r
 
t
i
p
s
.
 
M
o
t
h
e
r
 
W
o
l
f
 
l
a
y
 
w
i
t
h
 
h
e
r
 
b
i
g
 
g
r
a
y
 
n
o
s
e
 
d
r
o
p
p
e
d
 
a
c
r
o
s
s
 
h
e
r


f
o
u
r
 
t
u
m
b
l
i
n
g
,
 
s
q
u
e
a
l
i
n
g
 
c
u
b
s
,
 
a
n
d
 
t
h
e
 
m
o
o
n
 
s
h
o
n
e
 
i
n
t
o
 
t
h
e
 
m
o
u
t
h
 
o
f
 
t
h
e


c
a
v
e
 
w
h
e
r
e
 
t
h
e
y
 
a
l
l
 
l
i
v
e
d
.
 
“
A
u
g
r
h
!
”
 
s
a
i
d
 
F
a
t
h
e
r
 
W
o
l
f
.
 
“
I
t
 
i
s
 
t
i
m
e
 
t
o


h
u
n
t
 
a
g
a
i
n
.
”
 
H
e
 
w
a
s
 
g
o
i
n
g
 
t
o
 
s
p
r
i
n
g
 
d
o
w
n
 
h
i
l
l
 
w
h
e
n
 
a
 
l
i
t
t
l
e
 
s
h
a
d
o
w
 
w
i
t


In [21]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [22]:
sequences

<BatchDataset shapes: (101,), types: tf.int64>

In [23]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [24]:
dataset = sequences.map(create_seq_targets)

In [25]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[85 34 74  1 77 55 73  1 73 59 76 59 68  1 69 82 57 66 69 57 65  1 69 60
  1 55  1 76 59 72 79  1 77 55 72 67  1 59 76 59 68 63 68 61  1 63 68  1
 74 62 59  1 44 59 59 69 68 59 59  1 62 63 66 66 73  1 77 62 59 68  0 31
 55 74 62 59 72  1 48 69 66 60  1 77 69 65 59  1 75 70  1 60 72 69 67  1
 62 63 73  1]
﻿It was seven o’clock of a very warm evening in the Seeonee hills when
Father Wolf woke up from his 


[34 74  1 77 55 73  1 73 59 76 59 68  1 69 82 57 66 69 57 65  1 69 60  1
 55  1 76 59 72 79  1 77 55 72 67  1 59 76 59 68 63 68 61  1 63 68  1 74
 62 59  1 44 59 59 69 68 59 59  1 62 63 66 66 73  1 77 62 59 68  0 31 55
 74 62 59 72  1 48 69 66 60  1 77 69 65 59  1 75 70  1 60 72 69 67  1 62
 63 73  1 58]
It was seven o’clock of a very warm evening in the Seeonee hills when
Father Wolf woke up from his d


Generating training batches

In [26]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [27]:
dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

Create the Model

In [28]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Creating the function, just to set the "from_logits=True", as it cant be done when its called in the model.compile

In [29]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [30]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [31]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5504      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 86)           88322     
Total params: 3,455,002
Trainable params: 3,455,002
Non-trainable params: 0
_________________________________________________________________


Model Training

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 100, 86)  <=== (batch_size, sequence_length, vocab_size)


In [33]:
example_batch_predictions

<tf.Tensor: shape=(128, 100, 86), dtype=float32, numpy=
array([[[ 2.8023538e-03,  6.2504743e-04, -4.5671654e-03, ...,
          8.8604381e-03, -4.6302862e-03,  7.2382006e-04],
        [ 1.4996742e-03,  1.7696379e-04, -2.4503912e-03, ...,
          9.0468135e-03, -1.0047652e-02,  7.1900659e-03],
        [ 3.6090356e-03, -4.0994529e-03,  1.3138730e-03, ...,
          1.9106388e-04, -6.9942265e-03,  6.9705201e-03],
        ...,
        [ 7.7414261e-03,  3.4189899e-04, -8.2666110e-03, ...,
         -6.8813367e-03, -1.1341625e-02,  6.1693490e-03],
        [ 6.4810859e-03, -1.6488928e-03, -1.0044958e-03, ...,
         -8.7335156e-03, -7.4498975e-03,  4.8984191e-03],
        [ 1.2459241e-02,  3.7348983e-03,  4.6436801e-03, ...,
         -1.3962424e-02, -1.4315343e-03,  4.4790567e-03]],

       [[ 5.2096825e-03,  1.7620837e-03, -5.0730137e-03, ...,
         -6.0555032e-03, -4.9323016e-03,  2.1271913e-03],
        [ 4.8215119e-03, -5.0161383e-03, -1.0911295e-03, ...,
          1.2685149e-04, -2

In [34]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[15],
       [54],
       [38],
       [ 4],
       [50],
       [32],
       [30],
       [10],
       [59],
       [43],
       [85],
       [10],
       [50],
       [85],
       [10],
       [78],
       [15],
       [55],
       [30],
       [19],
       [38],
       [54],
       [70],
       [79],
       [55],
       [ 7],
       [56],
       [11],
       [37],
       [13],
       [22],
       [74],
       [77],
       [24],
       [15],
       [ 8],
       [53],
       [21],
       [40],
       [53],
       [41],
       [ 8],
       [61],
       [67],
       [67],
       [78],
       [69],
       [81],
       [13],
       [55],
       [68],
       [44],
       [85],
       [76],
       [20],
       [35],
       [77],
       [ 2],
       [80],
       [69],
       [42],
       [77],
       [75],
       [80],
       [50],
       [39],
       [ 4],
       [12],
       [52],
       [64],
       [82],
       [ 2],
       [58],
   

In [35]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([15, 54, 38,  4, 50, 32, 30, 10, 59, 43, 85, 10, 50, 85, 10, 78, 15,
       55, 30, 19, 38, 54, 70, 79, 55,  7, 56, 11, 37, 13, 22, 74, 77, 24,
       15,  8, 53, 21, 40, 53, 41,  8, 61, 67, 67, 78, 69, 81, 13, 55, 68,
       44, 85, 76, 20, 35, 77,  2, 80, 69, 42, 77, 75, 80, 50, 39,  4, 12,
       52, 64, 82,  2, 58, 66, 39, 85, 84, 23, 10, 66, 21, 65, 36, 26, 85,
       16, 10, 41, 60, 60, 41, 59, 56, 53, 77, 68, 58, 52, 67,  8])

In [36]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))

Given the input seq: 

rt at five feet, and bound round the ends, to prevent them
splitting, with bands of copper; but he c


Next Char Predictions: 

3`M%YGE.eR﻿.Y﻿.x3aE7M`pya*b/L1:tw?3,]9O]P,gmmxo‘1anS﻿v8Jw!zoQwuzYN%0[j’!dlN﻿”;.l9kKA﻿4.PffPeb]wnd[m,


In [37]:
epochs = 30
model.fit(dataset,epochs=epochs)

Epoch 1/30
22/22 [==============================] - 136s 6s/step - loss: 4.2626
Epoch 2/30
22/22 [==============================] - 136s 6s/step - loss: 3.1719
Epoch 3/30
22/22 [==============================] - 137s 6s/step - loss: 2.9972
Epoch 4/30
22/22 [==============================] - 137s 6s/step - loss: 2.7668
Epoch 5/30
22/22 [==============================] - 136s 6s/step - loss: 2.5748
Epoch 6/30
22/22 [==============================] - 136s 6s/step - loss: 2.4735
Epoch 7/30
22/22 [==============================] - 136s 6s/step - loss: 2.4027
Epoch 8/30
22/22 [==============================] - 135s 6s/step - loss: 2.3440
Epoch 9/30
22/22 [==============================] - 135s 6s/step - loss: 2.2851
Epoch 10/30
22/22 [==============================] - 136s 6s/step - loss: 2.2307
Epoch 11/30
22/22 [==============================] - 137s 6s/step - loss: 2.1783
Epoch 12/30
22/22 [==============================] - 141s 6s/step - loss: 2.1226
Epoch 13/30
22/22 [==================

In [39]:
model.save('/content/drive/My Drive/Colab Notebooks/TextFiles/lionking_gen.h5') 

In [40]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('/content/drive/My Drive/Colab Notebooks/TextFiles/lionking_gen.h5')

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5504      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 86)             88322     
Total params: 3,455,002
Trainable params: 3,455,002
Non-trainable params: 0
_________________________________________________________________


In [41]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

**Model geenrated data based on the seed word**

In [59]:
print(generate_text(model,"Mowgli ",gen_size=1000))

Mowgli that made to eath on the ground,
and if no wor!
     The moskets,” he said, for there who can to thucked atixald
to meat Nag?”

If ex the came aster that he had been heavy up the copyan in Indian from
People.” Changs and out of the iss to sty of callata! It!
     Averad a compenter day,
where has noow his because. He was ear wairounds a
           woll hinchess in the newty.”

Litten Wolfs, and Kaa. “A rile tonk of the Humpriwht wird out on the carnaingles, hundred feolless out of his fifter slamons, under ast araverwamnout bride
beastens ahe canching up nothing in the sefors without a
neatly as his line and mulitut. He wishout broods, vercops and stunched See threw the singer” as hed and as
of compliy
noubred man nater. Har! All they can out of anything bag into mistul, and broke, how to huntre the prass at here as explain, and
Rana.
Lowg Wolves where he seascopthela. It’s hone.-
“He had been saw ref on his shapped and diggling.

“Thatkien carcoser and
wampuntwer, shere Khan’s 